In [1]:
import tensorflow as tf
import numpy as np

# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but TensorFlow will
# figure that out for us.)
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for step in range(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

(0, array([-0.17580414], dtype=float32), array([ 0.70379657], dtype=float32))
(20, array([ 0.00500412], dtype=float32), array([ 0.35631824], dtype=float32))
(40, array([ 0.07632747], dtype=float32), array([ 0.31403425], dtype=float32))
(60, array([ 0.09410091], dtype=float32), array([ 0.30349728], dtype=float32))
(80, array([ 0.09852997], dtype=float32), array([ 0.30087152], dtype=float32))
(100, array([ 0.09963366], dtype=float32), array([ 0.30021718], dtype=float32))
(120, array([ 0.0999087], dtype=float32), array([ 0.30005413], dtype=float32))
(140, array([ 0.09997726], dtype=float32), array([ 0.30001348], dtype=float32))
(160, array([ 0.09999432], dtype=float32), array([ 0.30000338], dtype=float32))
(180, array([ 0.09999859], dtype=float32), array([ 0.30000085], dtype=float32))
(200, array([ 0.09999967], dtype=float32), array([ 0.30000022], dtype=float32))
